In [ ]:
# Useful imports:
from pathlib import Path

from qcodes.dataset import (
    Measurement,
    initialise_or_create_database_at,
    load_or_create_experiment,
    ThreadPoolParamsCaller,
    ArraySweep,
    dond,
    plot_dataset,
)
from qcodes.instrument_drivers.mock_instruments import DummyInstrument, DummyInstrumentWithMeasurement
from qcodes.parameters import Parameter
from qcodes.station import Station
import qcodes as qc
import numpy as np
import time
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

In [ ]:
station = Station(config_file=r"station_test.yaml")
station.load_all_instruments()

In [ ]:
dac = DummyInstrument("dac", gates=["ch1"])
dmm = DummyInstrumentWithMeasurement(name="dmm", setter_instr=dac)

In [ ]:
# Let's initialize a database to ensure that it exists
initialise_or_create_database_at(Path.cwd() / "snapshot_example.db")

# Let's create a new experiment
experiment = load_or_create_experiment("snapshot_experiment", "no_sample_yet")
measurement = Measurement(experiment, station)

In [ ]:
measurement.register_parameter(station.kiutra.temperature_control.setpoint)
measurement.register_parameter(dac.ch1)
measurement.register_parameter(station.keithley1.measure, setpoints=[station.kiutra.temperature_control.setpoint, dac.ch1])
measurement.register_parameter(station.kiutra.temperature_control.temperature, setpoints=[station.kiutra.temperature_control.setpoint, dac.ch1])
measurement.register_parameter(station.kiutra.magnet_control.field, setpoints=[station.kiutra.temperature_control.setpoint, dac.ch1])

In [ ]:
station.kiutra.temperature_control.is_stable() is False

In [ ]:
pool_caller = ThreadPoolParamsCaller(
    station.kiutra.temperature_control.setpoint,
    dac.ch1,
    station.kiutra.temperature_control.temperature,
    station.kiutra.magnet_control.field,
    station.keithley1.measure,
)

with (
    measurement.run() as datasaver,
    pool_caller as call_params_in_pool,
):
    for temp in np.linspace(2.4,2.6,3):
        station.kiutra.temperature_control.setpoint(temp)
        time.sleep(1)
        while station.kiutra.temperature_control.is_stable() is False:
            time.sleep(1)
        for gate in np.linspace(0, 1, 10):
            dac.ch1(gate)
            time.sleep(0.01)
            datasaver.add_result(*call_params_in_pool())
    # Convenient to have for plotting and data access
    dataset = datasaver.dataset

In [ ]:
plot_dataset(dataset)

In [ ]:
measurements = (
    station.keithley1.measure,
    station.kiutra.temperature_control.temperature,
    station.kiutra.magnet_control.field,
)
sweepZeroP = np.arange(0, 1, 0.01)
sweepPN = np.arange(1,-1,-0.01)
sweepNZero = np.arange(-1, 1e-12, 0.01)
sweep = np.concatenate((sweepZeroP, sweepPN, sweepNZero))
measurementSweep = ArraySweep(station.mfli1.sigouts[0].offset, sweep, delay=0.2, get_after_set=True)

In [ ]:
t0 = time.perf_counter()
dond(measurementSweep, *measurements, do_plot=True, use_threads=False)
t1 = time.perf_counter()

print("Report:")
print(f"Data acquisition time: {t1 - t0} s")